In [1]:
%pip install --user labelme tensorflow tensorflow-gpu opencv-python matplotlib albumentations split-folders imutils scikit-learn keras

  Using cached labelme-5.1.1-py3-none-any.whl
  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached albumentations-1.3.0-py3-none-any.whl (123 kB)
  Using cached split_folders-0.5.1-py3-none-any.whl (8.4 kB)
  Using cached natsort-8.3.1-py3-none-any.whl (38 kB)
  Using cached imgviz-1.7.2-py3-none-any.whl (7.7 MB)
  Using cached scikit_image-0.20.0-cp310-cp310-win_amd64.whl (23.7 MB)
  Using cached qudida-0.0.4-py3-none-any.whl (3.5 kB)
  Using cached lazy_loader-0.2-py3-none-any.whl (8.6 kB)
  Using cached networkx-3.0-py3-none-any.whl (2.0 MB)
     ---------------------------------------- 0.0/3.4 MB ? eta -:--:--
     ---- ----------------------------------- 0.4/3.4 MB 11.2 MB/s eta 0:00:01
     --------- ------------------------------ 0.8/3.4 MB 10.2 MB/s eta 0:00:01
     --------------- ------------------------ 1.3/3.4 MB 8.9 MB/s eta 0:00:01
     ------------------- 

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [18 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\Harry Parker\AppData\Local\Temp\pip-install-0j4gpn_y\tensorflow-gpu_484c3eb9cccb436c835ea81b7b9ed567\setup.py", line 37, in <module>
          raise Exception(TF_REMOVAL_WARNING)
      Exception:
      
      The "tensorflow-gpu" package has been removed!
      
      Please install "tensorflow" instead.
      
      Other than the name, the two packages have been identical
      since TensorFlow 2.1, or roughly since Sep 2019. For more
      information, see: pypi.org/project/tensorflow-gpu
      
      
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tensorflow-gpu
  error: subpr

In [2]:
import os
import cv2
import json

import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from scipy.io import loadmat



In [3]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [7]:
# Define the path to the dataset
data_path = "data/WIDERFace"
AUTOTUNE = 1000
batch_size = 32

# Define the paths to the train, test, and val sets
train_path = os.path.join(data_path, "WIDER_train/images")
test_path = os.path.join(data_path, "WIDER_test/images")
val_path = os.path.join(data_path, "WIDER_val/images")
labels_path = os.path.join(data_path, "wider_face_split")

# Define the paths to the ground truth text files
train_labels_path = os.path.join(train_path, "wider_face_train_bbx_gt.txt")
test_labels_path = os.path.join(test_path, "wider_face_test_filelist.txt")
val_labels_path = os.path.join(val_path, "wider_face_val_bbx_gt.txt")

# Define a generator function to load images
def load_images(image_paths):
    for image_path in image_paths:
        yield cv2.imread(image_path)

# Define a generator function to load labels
def load_labels(label_file_path):
    with open(label_file_path, "r") as f:
        lines = f.readlines()
        for line in lines:
            # process the line to extract the label information
            label_info = line.strip().split()
            yield label_info

# Load the train, test, and val images using the generator function
train_images = load_images((os.path.join(root, file) for root, dirs, files in os.walk(train_path) for file in files if file.endswith(".jpg")))
test_images = load_images((os.path.join(root, file) for root, dirs, files in os.walk(test_path) for file in files if file.endswith(".jpg")))
val_images = load_images((os.path.join(root, file) for root, dirs, files in os.walk(val_path) for file in files if file.endswith(".jpg")))

# Load the ground truth text files using the generator function
train_labels = load_labels(train_labels_path)
test_labels = load_labels(test_labels_path)
val_labels = load_labels(val_labels_path)

# Use TensorFlow's data pipeline to preprocess the data
train_image_paths = [os.path.join(root, file) for root, dirs, files in os.walk(train_path) for file in files if file.endswith(".jpg")]
train_data = tf.data.Dataset.from_generator(load_images, args=[train_image_paths], output_types=tf.uint8, output_shapes=tf.TensorShape([None, None, 3]))
train_data = train_data.map(lambda x: tf.image.convert_image_dtype(x, tf.float32))
train_labels = tf.data.Dataset.from_generator(load_labels, args=[train_labels_path], output_types=tf.float32)
train_dataset = tf.data.Dataset.zip((train_data, train_labels))
train_dataset = train_dataset.batch(batch_size).repeat().prefetch(buffer_size=AUTOTUNE)
steps_per_epoch = len(list((os.path.join(root, file) for root, dirs, files in os.walk(train_path) for file in files if file.endswith(".jpg")))) // batch_size

# Load and preprocess test data images
test_image_paths = [os.path.join(root, file) for root, dirs, files in os.walk(test_path) for file in files if file.endswith(".jpg")]
test_data = tf.data.Dataset.from_generator(load_images, args=[test_image_paths], output_types=tf.uint8, output_shapes=tf.TensorShape([None, None, 3]))
test_data = test_data.map(lambda x: tf.image.convert_image_dtype(x, tf.float32))

# Load test labels
test_labels = tf.data.Dataset.from_generator(load_labels, args=[test_labels_path], output_types=tf.float32)

# Concatenate test data and labels
test_dataset = tf.data.Dataset.concatenate(test_data, test_labels)


# Use TensorFlow's data pipeline to preprocess the data
val_image_paths = [os.path.join(root, file) for root, dirs, files in os.walk(val_path) for file in files if file.endswith(".jpg")]
val_data = tf.data.Dataset.from_generator(load_images, args=[val_image_paths], output_types=tf.uint8, output_shapes=tf.TensorShape([None, None, 3]))
val_data = val_data.map(lambda x: tf.image.convert_image_dtype(x, tf.float32))
val_labels = tf.data.Dataset.from_generator(load_labels, args=[val_labels_path], output_types=tf.float32)
val_dataset = tf.data.Dataset.zip((val_data, val_labels))
val_dataset = val_dataset.batch(batch_size).repeat().prefetch(buffer_size=AUTOTUNE)
steps_per_epoch = len(list((os.path.join(root, file) for root, dirs, files in os.walk(train_path) for file in files if file.endswith(".jpg")))) // batch_size


VGG16

In [6]:
from keras.models import Model
from keras.layers import Input, Conv2D, Dense, GlobalMaxPooling2D
from keras.applications import VGG16

# Load the pre-trained VGG16 model
vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

vgg.summary()

#building classifer 
def build_model():

    # Freeze the pre-trained layers of the VGG16 model
    for layer in vgg.layers:
        layer.trainable = False

    # Define the input tensor
    input_tensor = Input(shape=(224, 224, 3))

    # Connect the input tensor to the pre-trained VGG16 model
    model = VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)

    f1 = GlobalMaxPooling2D()(model)
    class1 = Dense(1024, activation='relu')(f1)
    class2 = Dense(1, activation='relu')(class1)

    f2 = GlobalMaxPooling2D()(model)
    regress1 = Dense(1024, activation='relu')(f2)
    regress2 = Dense(4, activation='relu')(regress1)

    faceDetectionModel = Model(inputs=input_tensor, outputs=[class2, regress2])

    return faceDetectionModel

faceDetectionModel = build_model()
faceDetectionModel.summary()

X, y = train.as_numpy_iterator().next()
X.shape

classes, coords = faceDetectionModel.predict(X)
classes, coords

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

NameError: name 'Sequential' is not defined

Losses

In [ ]:
faceDetectionModel.compile(loss=custom_loss, optimizer=tf.keras.optimizers.Adam())

In [ ]:
def custom_loss(y_true, y_pred):
    mask_shape = tf.shape(y_true)[:4]
    print("Mask",np.shape(mask_shape))

    cell_x = tf.to_float(tf.reshape(tf.tile(tf.range(GRID_W), [GRID_H]), (1, GRID_H, GRID_W, 1, 1)))
    print("cell_x",np.shape(cell_x))
    cell_y = cell_x
    print("cell_y",np.shape(cell_y))

    cell_grid = tf.tile(tf.concat([cell_x,cell_y], -1), [BATCH_SIZE, 1, 1, 5, 1])
    print("cell_grid",np.shape(cell_grid))

    coord_mask = tf.zeros(mask_shape)
    conf_mask  = tf.zeros(mask_shape)
    print("coord_mask",np.shape(coord_mask))
    
    seen = tf.Variable(0.)
    total_recall = tf.Variable(0.)
    
    """
    Adjust prediction
    """
    pred_box_xy = tf.sigmoid(y_pred[..., :2]) + cell_grid
    pred_box_wh = tf.exp(y_pred[..., 2:4]) * np.reshape(ANCHORS, [1,1,1,BOX,2])    
    pred_box_conf = tf.sigmoid(y_pred[..., 4])
    """
    Adjust ground truth
    """
    true_box_xy = y_true[..., 0:2] 
    true_box_wh = y_true[..., 2:4] 
    true_box_conf = y_true[..., 4]
    """
    Determine the masks
    """
    ### coordinate mask: simply the position of the ground truth boxes (the predictors)
    coord_mask = tf.expand_dims(y_true[..., 4], axis=-1) * COORD_SCALE
    """
    Warm-up training
    """
    no_boxes_mask = tf.to_float(coord_mask < COORD_SCALE/2.)
    
    seen = tf.assign_add(seen, 1.)
    true_box_xy, true_box_wh, coord_mask = tf.cond(tf.less(seen, WARM_UP_BATCHES),
                   lambda: [true_box_xy + (0.5) * no_boxes_mask, 
                   true_box_wh + tf.ones_like(true_box_wh) * np.reshape(ANCHORS, [1,1,1,BOX,2])* no_boxes_mask, tf.ones_like(coord_mask)],
                   lambda: [true_box_xy, true_box_wh, coord_mask])
    
 
    """
    Finalize the loss
    """
    nb_coord_box = tf.reduce_sum(tf.to_float(coord_mask > 0.0))
    # nb_conf_box  = tf.reduce_sum(tf.to_float(conf_mask  > 0.0))

    loss_xy    = tf.reduce_sum(tf.square(true_box_xy-pred_box_xy)     * coord_mask) / (nb_coord_box + 1e-6) / 2.
    loss_wh    = tf.reduce_sum(tf.square(tf.sqrt(true_box_wh)-tf.sqrt(pred_box_wh))     * coord_mask) / (nb_coord_box + 1e-6) / 2.
    loss_conf  = tf.reduce_sum(tf.square(true_box_conf-pred_box_conf))
    loss = loss_xy + loss_wh + loss_conf
    
    return loss

In [ ]:
class FaceDetection(Model):
      def __init__(self, facecctv, **kwargs):
        super().__init__(**kwargs)
        self.model = facecctv

      def compile(self, optimiser, classloss, localisationloss, **kwargs):
          super().compile(**kwargs)
          self.classloss = classloss
          self.localisationloss = localisationloss
          self.optimiser = optimiser
      
      def train_step(self, batch, **kwargs):
          
            X, y = batch

            with tf.GradientTape() as tape:
                  classes, coords = self.model(X, training=True)

                  batch_class_loss = self.classloss(y[0], classes)

                  batch_localisation_loss = self.localisationloss(tf.cast(y[1], tf.float32), coords)

                  total_loss = 0.5*batch_class_loss + batch_localisation_loss

                  gradients = tape.gradient(total_loss, self.model.trainable_variables)

            optimiser.apply_gradients(zip(gradients, self.model.trainable_variables))

            return {"class_loss": batch_class_loss, "localisation_loss": batch_localisation_loss, "total_loss": total_loss}
      
      def test_step(self, batch, **kwargs):
            X, y = batch

            classes, coords = self.model(X, training=False)

            batch_class_loss = self.classloss(y[0], classes)

            batch_localisation_loss = self.localisationloss(tf.cast(y[1], tf.float32), coords)

            total_loss = 0.5*batch_class_loss + batch_localisation_loss

            return {"class_loss": batch_class_loss, "localisation_loss": batch_localisation_loss, "total_loss": total_loss}
          
      def call(self, X, **kwargs):
            return self.model(X, **kwargs)
      
model = FaceDetection(faceDetectionModel)
model.compile(optimiser, classloss, regloss)

logdir = 'logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logdir, histogram_freq = 1)
hist = model.fit(train, epochs=10, validation_data=val, callbacks=[tensorboard_callback])

hist.history

fig, axes = plt.subplots(ncols=3, figsize=(20, 5))

axes[0].plot(hist.history['total_loss'], color='teal', label='total_loss')
axes[0].plot(hist.history['val_total_loss'], color='orange', label='val_total_loss')
axes[0].set_title('Total Loss')
axes[0].legend()

axes[1].plot(hist.history['class_loss'], color='teal', label='class_loss')
axes[1].plot(hist.history['val_class_loss'], color='orange', label='val_class_loss')
axes[1].set_title('Classification Loss')
axes[1].legend()

axes[2].plot(hist.history['localisation_loss'], color='teal', label='localisation_loss')
axes[2].plot(hist.history['val_localisation_loss'], color='orange', label='val_localisation_loss')
axes[2].set_title('Localisation Loss')
axes[2].legend()

plt.show()